<a href="https://colab.research.google.com/github/remphanstar/WanBook/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Cell 1 - WAN2GP Setup Introduction + Enhanced Platform Detection

import sys
from IPython.display import display, HTML, Markdown
import os

def jupyter_lab_title(title):
    if 'google.colab' not in sys.modules:
        display(Markdown(f"## {title}"))

jupyter_lab_title("WAN2GP Setup Introduction + Enhanced Platform Detection")

def detect_platform():
    """Enhanced platform detection with comprehensive indicators"""

    # Lightning AI detection - multiple indicators for reliability
    lightning_indicators = [
        'lightning' in str(sys.executable).lower(),
        'teamspace-studios' in os.getcwd(),
        'LIGHTNING_CLOUD_SPACE_HOST' in os.environ,
        'LIGHTNING_CLOUD_SPACE_ID' in os.environ,
        'commands/python' in str(sys.executable),
        '/home/zeus/miniconda3/envs/cloudspace' in str(sys.executable),
        os.path.exists('/teamspace'),
        os.path.exists('/commands')
    ]

    # Google Colab detection
    colab_indicators = [
        'google.colab' in sys.modules,
        '/content' in os.getcwd()
    ]

    # Vast.AI detection
    vast_indicators = [
        'VAST_CONTAINERLABEL' in os.environ,
        '/workspace' in os.getcwd(),
        'vast' in os.environ.get('HOSTNAME', '').lower()
    ]

    if any(lightning_indicators):
        return "Lightning AI"
    elif any(colab_indicators):
        return "Google Colab"
    elif any(vast_indicators):
        return "Vast.AI/Generic"
    else:
        return "Vast.AI/Generic"

def get_platform_commands(platform):
    """Get platform-specific pip and python commands"""
    if platform == "Lightning AI":
        return "pip", "python", False  # pip_cmd, python_cmd, use_venv
    elif platform == "Google Colab":
        return ".venv/bin/pip", ".venv/bin/python", True
    else:  # Vast.AI/Generic
        return ".venv/bin/pip", ".venv/bin/python", True

# Detect current platform
current_platform = detect_platform()
pip_cmd, python_cmd, use_venv = get_platform_commands(current_platform)

# Display platform information
display(HTML(f"""
<div style="font-family: Arial, sans-serif; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 20px; border-radius: 10px; text-align: center; margin: 10px 0; box-shadow: 0 4px 15px rgba(0,0,0,0.2);">
    <h1>🚀 Wan2GP Full Setup Notebook</h1>
    <p>A cross-platform Jupyter notebook (Colab / Lightning AI / Vast.ai) that installs Wan2GP, common LoRA packs, and optional performance extras (FlashAttention 2, SageAttention, xFormers). It accelerates all downloads with <strong>aria2c</strong> for maximum speed.</p>
</div>

<div style="background-color: #f8f9fa; padding: 20px; border-radius: 8px; border: 1px solid #dee2e6; margin: 15px 0;">
    <div style="background-color: #007bff; color: white; padding: 15px; border-radius: 5px; text-align: center; margin-bottom: 20px;">
        <h3 style="margin: 0; color: white;">🔍 Platform Detection Results</h3>
        <p style="margin: 5px 0 0 0; color: white;">Currently Running on: <strong style="color: #28a745; background-color: white; padding: 2px 6px; border-radius: 3px;">{current_platform}</strong></p>
        <p style="margin: 5px 0 0 0; color: white;">Virtual Environment: <strong>{"Yes" if use_venv else "No (Lightning AI)"}</strong></p>
        <p style="margin: 5px 0 0 0; color: white;">Pip Command: <strong>{pip_cmd}</strong></p>
        <p style="margin: 5px 0 0 0; color: white;">Python Command: <strong>{python_cmd}</strong></p>
    </div>
</div>
"""))

print("🔍 Platform Detection Debug Info:")
print(f"   - Detected Platform: {current_platform}")
print(f"   - Python Executable: {sys.executable}")
print(f"   - Current Working Directory: {os.getcwd()}")
print(f"   - Google Colab Check: {'google.colab' in sys.modules}")
print(f"   - Lightning Environment Variables: {[key for key in os.environ.keys() if 'LIGHTNING' in key]}")
print(f"   - Virtual Environment Usage: {use_venv}")
print(f"   - Pip Command: {pip_cmd}")
print(f"   - Python Command: {python_cmd}")


🔍 Platform Detection Debug Info:
   - Detected Platform: Google Colab
   - Python Executable: /usr/bin/python3
   - Current Working Directory: /content
   - Google Colab Check: True
   - Lightning Environment Variables: []
   - Virtual Environment Usage: True
   - Pip Command: .venv/bin/pip
   - Python Command: .venv/bin/python


In [ ]:
#@title Cell (2) Complete System + Python Environment Setup - Cross-Platform v3.1 (With Directory Protection)

import sys
import os
import subprocess
import shutil
import time
from pathlib import Path

def jupyter_lab_title(title):
    if "google.colab" not in sys.modules:
        from IPython.display import display, Markdown
        display(Markdown(f"## {title}"))

jupyter_lab_title("Complete System + Python Environment Setup")

class CompleteEnvironmentSetup:
    def __init__(self):
        # Use global platform configuration from Cell 1
        try:
            self.current_platform = current_platform
            self.pip_cmd = pip_cmd
            self.python_cmd = python_cmd
            self.use_venv = use_venv
        except NameError:
            print("⚠️ Platform detection variables not found. Using fallback detection.")
            self.detect_platform_fallback()

        self.setup_phases = [
            ("System Packages", self.install_system_packages),
            ("Directory Protection", self.setup_directory_protection),
            ("Repository Clone", self.clone_repository),
            ("Virtual Environment", self.setup_virtual_environment),
            ("PyTorch Installation", self.install_pytorch),
            ("Requirements", self.install_requirements),
            ("Environment Verification", self.verify_complete_setup),
        ]

        self.current_phase = 0
        self.total_phases = len(self.setup_phases)

    def detect_platform_fallback(self):
        """Fallback platform detection if Cell 1 wasn't run"""
        if "google.colab" in sys.modules:
            self.current_platform = "Google Colab"
            self.pip_cmd, self.python_cmd, self.use_venv = ".venv/bin/pip", ".venv/bin/python", True
        elif any(["lightning" in str(sys.executable).lower(), "teamspace" in os.getcwd()]):
            self.current_platform = "Lightning AI"
            self.pip_cmd, self.python_cmd, self.use_venv = "pip", "python", False
        else:
            self.current_platform = "Generic"
            self.pip_cmd, self.python_cmd, self.use_venv = ".venv/bin/pip", ".venv/bin/python", True

    def update_progress(self, phase_name, status="in_progress"):
        """Update setup progress with visual indicators"""
        progress_percent = (self.current_phase / self.total_phases) * 100
        status_icons = {
            "in_progress": "🔄",
            "success": "✅",
            "failed": "❌"
        }

        print(f"\n[{self.current_phase + 1}/{self.total_phases}] {status_icons[status]} {phase_name}")
        print(f"Progress: {progress_percent:.1f}% | Platform: {self.current_platform}")
        print("=" * 60)

    def run_command_safely(self, command, description, timeout=300):
        """Execute command with comprehensive error handling"""
        try:
            result = subprocess.run(
                command,
                capture_output=True,
                text=True,
                timeout=timeout
            )

            if result.returncode == 0:
                print(f"✅ {description} completed successfully")
                return True, result.stdout
            else:
                print(f"❌ {description} failed (exit code {result.returncode})")
                if result.stderr:
                    # Check for specific ensurepip failure
                    if "ensurepip" in result.stderr:
                        print(f"⚠️ ensurepip module failure detected")
                        return "ensurepip_failure", result.stderr
                    print(f"   Error: {result.stderr.strip()[:200]}")
                return False, result.stderr

        except subprocess.TimeoutExpired:
            print(f"⏰ {description} timed out after {timeout}s")
            return False, f"Timeout after {timeout}s"
        except Exception as e:
            print(f"💥 {description} crashed: {e}")
            return False, str(e)

    def install_system_packages(self):
        """Install system packages - same for all platforms"""
        print("🔧 Installing system packages...")

        if not shutil.which("apt-get"):
            print("📦 apt-get not found. Skipping system package installation.")
            return True

        try:
            print("📦 Updating package lists...")
            subprocess.run(["sudo", "apt-get", "update", "-qq"], check=True, timeout=60)

            print("🔧 Installing aria2, git, build-essential, and wget...")
            subprocess.run(["sudo", "apt-get", "install", "-y", "aria2", "git", "build-essential", "wget"], check=True, timeout=180)

            print("✅ System packages installed successfully")
            return True
        except subprocess.CalledProcessError as e:
            print(f"❌ System package installation failed: {e}")
            return False
        except subprocess.TimeoutExpired:
            print("⏰ System package installation timed out")
            return False

    def setup_directory_protection(self):
        """Critical directory protection to prevent recursive cloning"""
        print("🛡️ Setting up directory protection...")

        current_dir = os.getcwd()
        print(f"📍 Current directory: {current_dir}")

        # Check if we're already inside a Wan2GP directory
        if "Wan2GP" in current_dir:
            print("⚠️ Already inside Wan2GP directory - navigating to safe location")

            # Navigate to content root (Colab) or appropriate base directory
            if self.current_platform == "Google Colab":
                safe_dir = "/content"
            elif self.current_platform == "Lightning AI":
                safe_dir = os.path.expanduser("~")
            else:
                safe_dir = os.path.expanduser("~")

            try:
                os.chdir(safe_dir)
                print(f"📁 Moved to safe directory: {os.getcwd()}")
            except Exception as e:
                print(f"❌ Failed to navigate to safe directory: {e}")
                return False

        # Check for existing Wan2GP directory and handle it
        repo_path = os.path.join(os.getcwd(), "Wan2GP")
        if os.path.exists(repo_path):
            print(f"📁 Found existing Wan2GP directory at: {repo_path}")

            # Check if it looks like a valid repository
            if os.path.exists(os.path.join(repo_path, ".git")):
                print("✅ Existing directory appears to be a valid git repository")
                # Check if it's the correct repository
                try:
                    os.chdir(repo_path)
                    result = subprocess.run(["git", "remote", "get-url", "origin"],
                                          capture_output=True, text=True, timeout=10)
                    if result.returncode == 0 and "Wan2GP" in result.stdout:
                        print("✅ Existing repository is correct - using it")
                        return True
                    else:
                        print("⚠️ Existing repository is not the correct one")
                        os.chdir("..")
                except Exception as e:
                    print(f"⚠️ Could not verify existing repository: {e}")
                    os.chdir("..")

            # If we reach here, remove the problematic directory
            print("🗑️ Removing problematic existing directory...")
            try:
                shutil.rmtree(repo_path)
                print("✅ Problematic directory removed")
            except Exception as e:
                print(f"❌ Failed to remove existing directory: {e}")
                # Try to rename it instead
                try:
                    backup_name = f"Wan2GP_backup_{int(time.time())}"
                    os.rename(repo_path, backup_name)
                    print(f"✅ Renamed problematic directory to {backup_name}")
                except Exception as e2:
                    print(f"❌ Could not even rename directory: {e2}")
                    return False

        print("✅ Directory protection setup complete")
        return True

    def clone_repository(self):
        """Clone WAN2GP repository with enhanced protection"""
        REPO_DIR = "Wan2GP"
        REPO_URL = "https://github.com/deepbeepmeep/Wan2GP.git"

        print("📂 Setting up WAN2GP repository...")
        print(f"📍 Working from: {os.getcwd()}")

        # Final check - ensure we're not in a nested situation
        current_path = os.getcwd()
        if current_path.count("Wan2GP") > 0:
            print("🚨 CRITICAL: Still in Wan2GP directory path!")
            print("🔄 Attempting emergency navigation...")

            # Emergency navigation
            if self.current_platform == "Google Colab":
                emergency_path = "/content"
            else:
                emergency_path = os.path.expanduser("~")

            try:
                os.chdir(emergency_path)
                print(f"🆘 Emergency navigation successful: {os.getcwd()}")
            except Exception as e:
                print(f"💥 Emergency navigation failed: {e}")
                return False

        if not os.path.exists(REPO_DIR):
            print(f"📥 Cloning repository from {REPO_URL}...")
            try:
                subprocess.run(["git", "clone", "--depth", "1", REPO_URL], check=True, timeout=120)
                print("✅ Repository cloned successfully")
            except subprocess.CalledProcessError as e:
                print(f"❌ Failed to clone repository: {e}")
                return False
            except subprocess.TimeoutExpired:
                print("⏰ Repository clone timed out")
                return False
        else:
            print(f"📁 Directory {REPO_DIR} already exists")

        try:
            os.chdir(REPO_DIR)
            final_path = os.getcwd()
            print(f"📁 Changed to directory: {final_path}")

            # Verify we're in the right place
            if final_path.count("Wan2GP") > 1:
                print("🚨 WARNING: Detected nested Wan2GP directories!")
                print(f"🔍 Current path: {final_path}")
                # This shouldn't happen with our protection, but if it does, report it
                return False

            print("✅ Repository setup verified")
            return True
        except FileNotFoundError:
            print(f"❌ Directory {REPO_DIR} not found")
            return False

    def setup_venv_manual_pip(self):
        """Create venv without pip, then install pip manually"""
        try:
            print("🔧 Creating venv without pip...")
            subprocess.run([sys.executable, "-m", "venv", ".venv", "--without-pip"],
                          check=True, timeout=60)

            print("📥 Downloading get-pip.py...")
            subprocess.run(["wget", "https://bootstrap.pypa.io/get-pip.py"], check=True)

            print("🔧 Installing pip manually...")
            subprocess.run([".venv/bin/python", "get-pip.py"], check=True)

            print("✅ Virtual environment with manual pip created successfully")
            return True
        except Exception as e:
            print(f"❌ Manual pip installation failed: {e}")
            return False

    def setup_virtual_environment(self):
        """Create virtual environment ONLY if not on Lightning AI - Enhanced with ensurepip fixes"""
        if not self.use_venv:
            print("⚡ Lightning AI detected - using system environment (no venv)")
            return True

        print("🐍 Setting up virtual environment...")

        # Check if venv already exists
        if os.path.exists(".venv/bin/python"):
            print("✅ Virtual environment already exists and appears functional")
            return True

        print("🔨 Creating virtual environment...")

        # Try multiple methods for virtual environment creation
        venv_methods = [
            ([sys.executable, "-m", "venv", "--system-site-packages", ".venv"], "venv with system packages"),
            ([sys.executable, "-m", "venv", ".venv"], "standard venv"),
            ([sys.executable, "-m", "virtualenv", ".venv"], "virtualenv package"),
        ]

        for command, method_name in venv_methods:
            print(f"🔄 Trying: {method_name}")
            success, error_msg = self.run_command_safely(command, f"Virtual environment creation ({method_name})", timeout=60)

            if success == True:
                print("✅ Virtual environment created successfully")
                return True
            elif success == "ensurepip_failure":
                print("⚠️ ensurepip failure detected - trying manual pip installation...")
                if self.setup_venv_manual_pip():
                    return True
                break  # Don't try other methods if ensurepip failed
            else:
                print(f"❌ {method_name} failed")
                if error_msg:
                    print(f"   Error: {error_msg[:200]}")
                continue

        # Final fallback: override to system installation
        print("🔄 All virtual environment methods failed - switching to system installation")
        print("⚡ Overriding to system-wide installation (recommended for Colab)")
        self.use_venv = False
        self.pip_cmd = "pip"
        self.python_cmd = "python"
        return True

    def install_pytorch(self):
        """Install PyTorch with appropriate CUDA version"""
        print("🔥 Installing PyTorch...")

        pytorch_cmd = [
            self.pip_cmd, "install",
            "torch==2.6.0", "torchvision", "torchaudio",
            "--index-url", "https://download.pytorch.org/whl/test/cu124"
        ]

        success, _ = self.run_command_safely(pytorch_cmd, "PyTorch installation", timeout=300)
        return success == True

    def install_requirements(self):
        """Install requirements.txt"""
        print("📦 Installing requirements...")

        if not os.path.exists("requirements.txt"):
            print("⚠️ requirements.txt not found, skipping")
            return True

        req_cmd = [self.pip_cmd, "install", "-r", "requirements.txt"]
        success, _ = self.run_command_safely(req_cmd, "Requirements installation", timeout=300)
        return success == True

    def verify_complete_setup(self):
        """Verify the complete installation"""
        print("🔍 Verifying installation...")

        # Verify directory structure
        current_dir = os.getcwd()
        print(f"📍 Final directory: {current_dir}")

        if current_dir.count("Wan2GP") > 1:
            print("🚨 CRITICAL: Nested directory structure detected!")
            print("❌ Setup verification failed due to directory structure")
            return False

        try:
            # Test PyTorch installation
            result = subprocess.run([self.python_cmd, "-c", "import torch; print(f'PyTorch {torch.__version__} installed'); print(f'CUDA available: {torch.cuda.is_available()}')"],
                                  capture_output=True, text=True, timeout=30)

            if result.returncode == 0:
                print("✅ PyTorch verification successful:")
                for line in result.stdout.strip().split('\n'):
                    print(f"   {line}")

                # Verify main WanGP file exists
                if os.path.exists("wgp.py"):
                    print("✅ WanGP main file found")
                else:
                    print("⚠️ WanGP main file (wgp.py) not found")

                return True
            else:
                print(f"❌ PyTorch verification failed: {result.stderr}")
                return False

        except Exception as e:
            print(f"❌ Verification failed: {e}")
            return False

    def run_complete_setup(self):
        """Execute the complete setup process"""
        print("🚀 WAN2GP Complete System + Python Environment Setup")
        print("=" * 70)
        print(f"Platform: {self.current_platform}")
        print(f"Virtual Environment: {'Yes' if self.use_venv else 'No'}")
        print(f"Pip Command: {self.pip_cmd}")
        print(f"Python Command: {self.python_cmd}")
        print("=" * 70)

        start_time = time.time()

        for phase_name, phase_func in self.setup_phases:
            self.update_progress(phase_name, "in_progress")

            phase_start = time.time()
            success = phase_func()
            phase_duration = time.time() - phase_start

            if success:
                self.update_progress(phase_name, "success")
                print(f"⏱️ {phase_name} completed in {phase_duration:.1f}s")
                self.current_phase += 1
            else:
                self.update_progress(phase_name, "failed")
                print(f"💥 Setup failed at: {phase_name}")
                print("\n🔄 Setup Issues - Platform-Specific Troubleshooting:")
                print("📱 Colab: Restart runtime and re-run all cells")
                print("⚡ Lightning.AI: Ensure you're using the correct Python version")
                print("🌌 Vast.AI: Check GPU drivers and CUDA installation")
                return False

        total_duration = time.time() - start_time
        print(f"\n🎉 WAN2GP setup completed successfully in {total_duration:.1f}s!")
        print("🚀 Ready to launch WAN2GP!")
        return True

# Execute the complete setup
setup = CompleteEnvironmentSetup()
setup.run_complete_setup()


🚀 WAN2GP Complete System + Python Environment Setup
Platform: Google Colab
Virtual Environment: Yes
Pip Command: .venv/bin/pip
Python Command: .venv/bin/python

[1/6] 🔄 System Packages
Progress: 0.0% | Platform: Google Colab
🔧 Installing system packages...
📦 Updating package lists...
🔧 Installing aria2, git, build-essential, and wget...
✅ System packages installed successfully

[1/6] ✅ System Packages
Progress: 0.0% | Platform: Google Colab
⏱️ System Packages completed in 19.7s

[2/6] 🔄 Repository Clone
Progress: 16.7% | Platform: Google Colab
📂 Setting up WAN2GP repository...
📥 Cloning repository from https://github.com/deepbeepmeep/Wan2GP.git...
✅ Repository cloned successfully
📁 Changed to directory: /content/Wan2GP

[2/6] ✅ Repository Clone
Progress: 16.7% | Platform: Google Colab
⏱️ Repository Clone completed in 1.3s

[3/6] 🔄 Virtual Environment
Progress: 33.3% | Platform: Google Colab
🐍 Setting up virtual environment...
🔨 Creating virtual environment...
🔄 Trying: venv with system

In [ ]:
#@title Cell (3) Complete Model & LoRA Downloads - Full Featured UI with HuggingFace Auth v3.0

import os
import subprocess
import sys
import time
import json
from pathlib import Path
from IPython.display import display, HTML, Markdown, Javascript
import requests
from urllib.parse import urlparse

def jupyter_lab_title(title):
    if "google.colab" not in sys.modules:
        display(Markdown(f"## {title}"))

jupyter_lab_title("Complete Model & LoRA Downloads - Full Featured UI")

# HuggingFace Authentication Setup
class HuggingFaceAuth:
    def __init__(self):
        self.token = None
        self.authenticated = False
        self.setup_auth_ui()

    def setup_auth_ui(self):
        """Setup HuggingFace authentication with rich UI"""
        auth_html = f"""
        <div style="font-family: Arial, sans-serif; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    color: white; padding: 15px; border-radius: 10px; margin: 10px 0;
                    box-shadow: 0 4px 15px rgba(0,0,0,0.2);">
            <h3 style="margin: 0; color: white;">🔐 HuggingFace Authentication</h3>
            <p style="margin: 5px 0;">Optional: Login for private model access and faster downloads</p>
            <div id="hf-auth-status" style="margin-top: 10px;">
                <button onclick="loginHuggingFace()"
                        style="background: #28a745; color: white; border: none; padding: 8px 16px;
                               border-radius: 5px; cursor: pointer; margin-right: 10px;">
                    🔑 Login to HuggingFace
                </button>
                <button onclick="skipAuth()"
                        style="background: #6c757d; color: white; border: none; padding: 8px 16px;
                               border-radius: 5px; cursor: pointer;">
                    ⏭️ Skip Authentication
                </button>
            </div>
            <div id="auth-result" style="margin-top: 10px; display: none;"></div>
        </div>
        """

        auth_js = """
        <script>
        function loginHuggingFace() {
            const authDiv = document.getElementById('hf-auth-status');
            authDiv.innerHTML = `
                <input type="password" id="hf-token" placeholder="Enter HuggingFace Token"
                       style="padding: 8px; border-radius: 5px; border: 1px solid #ccc; margin-right: 10px; width: 250px;">
                <button onclick="authenticateToken()"
                        style="background: #007bff; color: white; border: none; padding: 8px 16px;
                               border-radius: 5px; cursor: pointer;">
                    ✅ Authenticate
                </button>
                <p style="font-size: 12px; margin-top: 5px;">
                    Get your token from: <a href="https://huggingface.co/settings/tokens" target="_blank"
                    style="color: #ffc107;">https://huggingface.co/settings/tokens</a>
                </p>
            `;
        }

        function authenticateToken() {
            const token = document.getElementById('hf-token').value;
            if (token) {
                google.colab.kernel.invokeFunction('authenticate_hf', [token], {});
                document.getElementById('auth-result').style.display = 'block';
                document.getElementById('auth-result').innerHTML =
                    '<p style="color: #28a745;">🔑 Authenticating with HuggingFace...</p>';
            }
        }

        function skipAuth() {
            google.colab.kernel.invokeFunction('skip_hf_auth', [], {});
            document.getElementById('hf-auth-status').innerHTML =
                '<p style="color: #ffc107;">⏭️ Skipped HuggingFace authentication - using public access only</p>';
        }
        </script>
        """

        display(HTML(auth_html + auth_js))

    def authenticate(self, token):
        """Authenticate with HuggingFace"""
        try:
            import huggingface_hub
            huggingface_hub.login(token=token, write_permission=True)
            self.token = token
            self.authenticated = True
            self.show_auth_success()
            return True
        except Exception as e:
            self.show_auth_error(str(e))
            return False

    def show_auth_success(self):
        """Show authentication success"""
        success_html = """
        <div style="background: #d4edda; color: #155724; padding: 10px; border-radius: 5px;
                    border: 1px solid #c3e6cb; margin: 5px 0;">
            <strong>✅ HuggingFace Authentication Successful!</strong><br>
            🚀 You now have access to private models and faster download speeds
        </div>
        """
        display(HTML(success_html))

    def show_auth_error(self, error):
        """Show authentication error"""
        error_html = f"""
        <div style="background: #f8d7da; color: #721c24; padding: 10px; border-radius: 5px;
                    border: 1px solid #f5c6cb; margin: 5px 0;">
            <strong>❌ Authentication Failed:</strong><br>
            {error}<br>
            <small>Continuing with public access only...</small>
        </div>
        """
        display(HTML(error_html))

# Interactive Download System with Rich UI
class EnhancedModelDownloader:
    def __init__(self):
        # Use global platform configuration from previous cells
        try:
            self.current_platform = current_platform
            self.pip_cmd = pip_cmd
            self.python_cmd = python_cmd
            self.use_venv = use_venv
        except NameError:
            print("⚠️ Platform detection variables not found. Using fallback detection.")
            self.detect_platform_fallback()

        # Initialize HuggingFace auth
        self.hf_auth = HuggingFaceAuth()

        # Fixed URLs - Using correct repository name with underscore
        self.base_repo_url = "https://huggingface.co/Kijai/WanVideo_comfy"
        self.resolve_url = f"{self.base_repo_url}/resolve/main"

        # Download methods with authentication support
        self.download_methods = [
            ("aria2c", self.download_with_aria2c),
            ("wget", self.download_with_wget),
            ("python", self.download_with_python),
            ("hf_hub", self.download_with_hf_hub)
        ]

        # Enhanced model definitions
        self.setup_model_definitions()
        self.setup_ui_components()

        # Download statistics
        self.download_stats = {
            "total_files": 0,
            "downloaded_files": 0,
            "failed_files": 0,
            "total_size": 0,
            "downloaded_size": 0
        }

    def detect_platform_fallback(self):
        """Fallback platform detection if previous cells weren't run"""
        if "google.colab" in sys.modules:
            self.current_platform = "Google Colab"
            self.pip_cmd, self.python_cmd, self.use_venv = ".venv/bin/pip", ".venv/bin/python", True
        elif any(["lightning" in str(sys.executable).lower(), "teamspace" in os.getcwd()]):
            self.current_platform = "Lightning AI"
            self.pip_cmd, self.python_cmd, self.use_venv = "pip", "python", False
        else:
            self.current_platform = "Generic"
            self.pip_cmd, self.python_cmd, self.use_venv = ".venv/bin/pip", ".venv/bin/python", True

    def setup_model_definitions(self):
        """Setup enhanced model definitions with authentication support"""
        self.models = {
            "wan_14b_fp16": {
                "url": f"{self.resolve_url}/Wan2_1-T2V-14B_fp16.safetensors",
                "filename": "Wan2_1-T2V-14B_fp16.safetensors",
                "size": "29.1 GB",
                "path": "ckpts/",
                "requires_auth": False,
                "description": "Wan 2.1 14B model - Highest quality text-to-video generation"
            },
            "wan_14b_fp8": {
                "url": f"{self.resolve_url}/Wan2_1-T2V-14B_fp8_e4m3fn.safetensors",
                "filename": "Wan2_1-T2V-14B_fp8_e4m3fn.safetensors",
                "size": "14.6 GB",
                "path": "ckpts/",
                "requires_auth": False,
                "description": "Wan 2.1 14B FP8 quantized - Smaller size, minimal quality loss"
            },
            "wan_1_3b_fp32": {
                "url": f"{self.resolve_url}/Wan2_1-T2V-1_3B_fp32.safetensors",
                "filename": "Wan2_1-T2V-1_3B_fp32.safetensors",
                "size": "2.7 GB",
                "path": "ckpts/",
                "requires_auth": False,
                "description": "Wan 2.1 1.3B model - Fast generation for lower-end hardware"
            },
            "phantom_wan_14b": {
                "url": f"{self.resolve_url}/Phantom-Wan-14B_fp16.safetensors",
                "filename": "Phantom-Wan-14B_fp16.safetensors",
                "size": "29.1 GB",
                "path": "ckpts/",
                "requires_auth": False,
                "description": "Phantom model - Excellent for object/person transfer"
            }
        }

        self.text_encoders = {
            "umt5_xxl_fp16": {
                "url": f"{self.resolve_url}/umt5_xxl_fp16.safetensors",
                "filename": "umt5_xxl_fp16.safetensors",
                "size": "4.9 GB",
                "path": "text_encoders/",
                "requires_auth": False,
                "description": "T5 text encoder - Required for all video generation"
            }
        }

        self.vaes = {
            "wan_vae_bf16": {
                "url": f"{self.resolve_url}/Wan2_1_VAE_bf16.safetensors",
                "filename": "Wan2_1_VAE_bf16.safetensors",
                "size": "167 MB",
                "path": "vae/",
                "requires_auth": False,
                "description": "Video VAE - Converts between pixel and latent space"
            }
        }

        self.loras = {
            "causVid_14b": {
                "url": f"{self.resolve_url}/Wan21_CausVid_14B_T2V_lora_rank32.safetensors",
                "filename": "Wan21_CausVid_14B_T2V_lora_rank32.safetensors",
                "size": "201 MB",
                "path": "loras14B/",
                "requires_auth": False,
                "description": "CausVid LoRA - 4-12 step generation with 2x speed improvement"
            },
            "accVid_t2v_14b": {
                "url": f"{self.resolve_url}/Wan21AccVidT2V14Blorarank32fp16.safetensors",
                "filename": "Wan21AccVidT2V14Blorarank32fp16.safetensors",
                "size": "201 MB",
                "path": "loras14B/",
                "requires_auth": False,
                "description": "AccVid T2V LoRA - 2x speed with no classifier-free guidance"
            }
        }

    def setup_ui_components(self):
        """Setup rich UI components with CSS and JavaScript"""
        ui_css = """
        <style>
        .download-container {
            font-family: 'Segoe UI', Arial, sans-serif;
            max-width: 1200px;
            margin: 0 auto;
        }

        .header-gradient {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 20px;
            border-radius: 10px;
            text-align: center;
            margin: 10px 0;
            box-shadow: 0 4px 15px rgba(0,0,0,0.2);
        }

        .category-card {
            background: #f8f9fa;
            border: 1px solid #dee2e6;
            border-radius: 8px;
            padding: 20px;
            margin: 15px 0;
            transition: all 0.3s ease;
        }

        .category-card:hover {
            box-shadow: 0 4px 12px rgba(0,0,0,0.1);
            transform: translateY(-2px);
        }

        .category-header {
            background: #007bff;
            color: white;
            padding: 10px 15px;
            border-radius: 5px;
            margin-bottom: 15px;
            font-weight: bold;
        }

        .model-item {
            background: white;
            border: 1px solid #e9ecef;
            border-radius: 5px;
            padding: 15px;
            margin: 10px 0;
            display: flex;
            justify-content: space-between;
            align-items: center;
        }

        .model-info {
            flex-grow: 1;
        }

        .model-name {
            font-weight: bold;
            color: #333;
            margin-bottom: 5px;
        }

        .model-description {
            color: #666;
            font-size: 0.9em;
            margin-bottom: 5px;
        }

        .model-size {
            color: #007bff;
            font-weight: bold;
        }

        .download-btn {
            background: #28a745;
            color: white;
            border: none;
            padding: 8px 16px;
            border-radius: 5px;
            cursor: pointer;
            margin-left: 10px;
            transition: background 0.3s ease;
        }

        .download-btn:hover {
            background: #218838;
        }

        .download-btn:disabled {
            background: #6c757d;
            cursor: not-allowed;
        }

        .progress-container {
            background: #f8f9fa;
            border-radius: 8px;
            padding: 20px;
            margin: 20px 0;
            border: 1px solid #dee2e6;
        }

        .progress-bar {
            width: 100%;
            height: 20px;
            background: #e9ecef;
            border-radius: 10px;
            overflow: hidden;
            margin: 10px 0;
        }

        .progress-fill {
            height: 100%;
            background: linear-gradient(90deg, #28a745, #20c997);
            transition: width 0.3s ease;
            border-radius: 10px;
        }

        .stats-grid {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 15px;
            margin: 20px 0;
        }

        .stat-card {
            background: white;
            padding: 15px;
            border-radius: 8px;
            text-align: center;
            border: 1px solid #dee2e6;
        }

        .stat-number {
            font-size: 2em;
            font-weight: bold;
            color: #007bff;
        }

        .stat-label {
            color: #666;
            margin-top: 5px;
        }
        </style>
        """

        ui_js = """
        <script>
        class DownloadProgressManager {
            constructor() {
                this.activeDownloads = new Map();
                this.totalProgress = 0;
            }

            startDownload(filename, size) {
                this.activeDownloads.set(filename, {
                    progress: 0,
                    size: size,
                    startTime: Date.now()
                });
                this.updateUI();
            }

            updateProgress(filename, progress) {
                if (this.activeDownloads.has(filename)) {
                    this.activeDownloads.get(filename).progress = progress;
                    this.updateUI();
                }
            }

            completeDownload(filename) {
                this.activeDownloads.delete(filename);
                this.updateUI();
            }

            updateUI() {
                // Update progress bars and statistics
                const totalDownloads = this.activeDownloads.size;
                const completedProgress = Array.from(this.activeDownloads.values())
                    .reduce((sum, download) => sum + download.progress, 0);

                const overallProgress = totalDownloads > 0 ? completedProgress / totalDownloads : 0;

                const progressBar = document.querySelector('.progress-fill');
                if (progressBar) {
                    progressBar.style.width = `${overallProgress}%`;
                }

                // Update statistics
                this.updateStats();
            }

            updateStats() {
                const statsContainer = document.querySelector('.stats-grid');
                if (statsContainer) {
                    const activeCount = this.activeDownloads.size;
                    const totalSize = Array.from(this.activeDownloads.values())
                        .reduce((sum, download) => sum + download.size, 0);

                    statsContainer.innerHTML = `
                        <div class="stat-card">
                            <div class="stat-number">${activeCount}</div>
                            <div class="stat-label">Active Downloads</div>
                        </div>
                        <div class="stat-card">
                            <div class="stat-number">${(totalSize / 1024 / 1024 / 1024).toFixed(1)}GB</div>
                            <div class="stat-label">Total Size</div>
                        </div>
                    `;
                }
            }
        }

        // Global download manager
        window.downloadManager = new DownloadProgressManager();

        // Download control functions
        function startDownload(modelKey, filename, size) {
            window.downloadManager.startDownload(filename, size);
            google.colab.kernel.invokeFunction('download_model', [modelKey], {});
        }

        function pauseDownload(filename) {
            console.log('Pausing download:', filename);
            // Implementation for pausing downloads
        }

        function resumeDownload(filename) {
            console.log('Resuming download:', filename);
            // Implementation for resuming downloads
        }
        </script>
        """

        display(HTML(ui_css + ui_js))

    def create_enhanced_ui(self):
        """Create enhanced download UI with rich components"""
        header_html = """
        <div class="download-container">
            <div class="header-gradient">
                <h1>🚀 WAN2GP Model & LoRA Download Center</h1>
                <p>Complete model collection with intelligent downloading and progress tracking</p>
                <div style="background: rgba(255,255,255,0.1); padding: 10px; border-radius: 5px; margin-top: 10px;">
                    <strong>Platform:</strong> {platform} |
                    <strong>Download Methods:</strong> aria2c → wget → python → HuggingFace Hub
                </div>
            </div>

            <div class="progress-container">
                <h3>📊 Download Progress</h3>
                <div class="progress-bar">
                    <div class="progress-fill" style="width: 0%;"></div>
                </div>
                <div class="stats-grid">
                    <div class="stat-card">
                        <div class="stat-number">0</div>
                        <div class="stat-label">Active Downloads</div>
                    </div>
                    <div class="stat-card">
                        <div class="stat-number">0GB</div>
                        <div class="stat-label">Total Size</div>
                    </div>
                </div>
            </div>
        """.format(platform=self.current_platform)

        # Generate category cards
        categories = [
            ("🎬 Essential Models", self.models, "Core video generation models"),
            ("📝 Text Encoders", self.text_encoders, "Required for text understanding"),
            ("🎨 VAE Models", self.vaes, "Video encoding/decoding components"),
            ("⚡ Performance LoRAs", self.loras, "Speed and quality enhancement LoRAs")
        ]

        category_html = ""
        for category_name, category_items, category_desc in categories:
            category_html += f"""
            <div class="category-card">
                <div class="category-header">
                    {category_name}
                    <div style="font-weight: normal; font-size: 0.9em; margin-top: 5px;">{category_desc}</div>
                </div>
            """

            for item_key, item_info in category_items.items():
                auth_indicator = "🔒" if item_info.get("requires_auth", False) else "🌐"
                category_html += f"""
                <div class="model-item">
                    <div class="model-info">
                        <div class="model-name">{auth_indicator} {item_info['filename']}</div>
                        <div class="model-description">{item_info.get('description', 'No description available')}</div>
                        <div class="model-size">Size: {item_info['size']}</div>
                    </div>
                    <button class="download-btn" onclick="startDownload('{item_key}', '{item_info['filename']}', {self.parse_size_to_bytes(item_info['size'])})">
                        📥 Download
                    </button>
                </div>
                """

            category_html += "</div>"

        # Control panel
        control_html = """
            <div class="category-card">
                <div class="category-header">🎮 Download Control Panel</div>
                <div style="display: flex; gap: 10px; flex-wrap: wrap;">
                    <button class="download-btn" onclick="downloadAll()">📦 Download All Essential</button>
                    <button class="download-btn" onclick="downloadModelsOnly()">🎬 Models Only</button>
                    <button class="download-btn" onclick="downloadLoRAsOnly()">⚡ LoRAs Only</button>
                    <button class="download-btn" style="background: #dc3545;" onclick="pauseAll()">⏸️ Pause All</button>
                    <button class="download-btn" style="background: #ffc107; color: black;" onclick="showAdvanced()">⚙️ Advanced Options</button>
                </div>
            </div>
        </div>
        """

        full_html = header_html + category_html + control_html

        # Enhanced JavaScript for download control
        enhanced_js = """
        <script>
        function downloadAll() {
            console.log('Starting download of all essential components...');
            google.colab.kernel.invokeFunction('download_all_essential', [], {});
        }

        function downloadModelsOnly() {
            console.log('Starting download of models only...');
            google.colab.kernel.invokeFunction('download_models_only', [], {});
        }

        function downloadLoRAsOnly() {
            console.log('Starting download of LoRAs only...');
            google.colab.kernel.invokeFunction('download_loras_only', [], {});
        }

        function pauseAll() {
            console.log('Pausing all downloads...');
            // Implementation for pausing all downloads
        }

        function showAdvanced() {
            console.log('Showing advanced options...');
            // Implementation for advanced options
        }
        </script>
        """

        display(HTML(full_html + enhanced_js))

    def parse_size_to_bytes(self, size_str):
        """Parse size string to bytes for JavaScript"""
        try:
            size_parts = size_str.split()
            if len(size_parts) >= 2:
                number = float(size_parts[0])
                unit = size_parts[1].upper()
                multipliers = {"GB": 1024**3, "MB": 1024**2, "KB": 1024}
                return int(number * multipliers.get(unit, 1))
        except:
            return 0
        return 0

    def download_with_hf_hub(self, url, filename, output_dir):
        """Download using HuggingFace Hub API with authentication"""
        try:
            if not self.hf_auth.authenticated:
                return False

            from huggingface_hub import hf_hub_download

            # Parse HuggingFace URL
            parsed_url = urlparse(url)
            path_parts = parsed_url.path.strip('/').split('/')

            if len(path_parts) >= 4:
                repo_id = f"{path_parts[0]}/{path_parts[1]}"
                file_path = "/".join(path_parts[3:])

                print(f"📥 Downloading with HuggingFace Hub API...")
                downloaded_file = hf_hub_download(
                    repo_id=repo_id,
                    filename=file_path,
                    token=self.hf_auth.token,
                    local_dir=output_dir,
                    local_dir_use_symlinks=False
                )

                print(f"✅ HuggingFace Hub download successful")
                return True

        except Exception as e:
            print(f"❌ HuggingFace Hub download failed: {e}")
            return False

    def download_with_aria2c(self, url, filename, output_dir):
        """Download using aria2c with authentication headers"""
        try:
            output_path = os.path.join(output_dir, filename)

            cmd = [
                "aria2c",
                "-x", "16",
                "-s", "16",
                "-k", "1M",
                "--continue",
                "--dir", output_dir,
                "--out", filename,
                url
            ]

            # Add authentication headers if available
            if self.hf_auth.authenticated:
                cmd.extend(["--header", f"Authorization: Bearer {self.hf_auth.token}"])

            print(f"📥 Downloading with aria2c (16 connections)...")
            result = subprocess.run(cmd, capture_output=True, text=True, timeout=1800)

            if result.returncode == 0:
                print(f"✅ aria2c download successful")
                return True
            else:
                print(f"❌ aria2c failed: {result.stderr[:200]}")
                return False

        except Exception as e:
            print(f"❌ aria2c error: {e}")
            return False

    def download_with_wget(self, url, filename, output_dir):
        """Download using wget with authentication"""
        try:
            output_path = os.path.join(output_dir, filename)

            cmd = [
                "wget",
                "--continue",
                "--progress=bar",
                "--timeout=30",
                "--tries=3",
                "-O", output_path,
                url
            ]

            # Add authentication headers if available
            if self.hf_auth.authenticated:
                cmd.extend(["--header", f"Authorization: Bearer {self.hf_auth.token}"])

            print(f"📥 Downloading with wget...")
            result = subprocess.run(cmd, capture_output=True, text=True, timeout=1800)

            if result.returncode == 0:
                print(f"✅ wget download successful")
                return True
            else:
                print(f"❌ wget failed: {result.stderr[:200]}")
                return False

        except Exception as e:
            print(f"❌ wget error: {e}")
            return False

    def download_with_python(self, url, filename, output_dir):
        """Download using Python urllib with authentication"""
        try:
            import urllib.request
            import shutil

            output_path = os.path.join(output_dir, filename)

            print(f"📥 Downloading with Python urllib...")

            # Create request with authentication if available
            req = urllib.request.Request(url)
            if self.hf_auth.authenticated:
                req.add_header('Authorization', f'Bearer {self.hf_auth.token}')

            with urllib.request.urlopen(req) as response:
                with open(output_path, 'wb') as out_file:
                    shutil.copyfileobj(response, out_file)

            print(f"✅ Python download successful")
            return True

        except Exception as e:
            print(f"❌ Python download failed: {e}")
            return False

    def create_directory_structure(self):
        """Create necessary directories for downloads"""
        directories = [
            "ckpts", "text_encoders", "vae", "vae_approx", "clip_vision",
            "loras", "loras1.3B", "loras14B", "lorasi2v", "lorashunyuan", "lorasltxv"
        ]

        print("📁 Creating directory structure...")
        for directory in directories:
            Path(directory).mkdir(parents=True, exist_ok=True)
            print(f"   ✅ {directory}/")

        print("✅ Directory structure created successfully")

    def run_enhanced_download_system(self):
        """Run the enhanced download system with full UI"""
        # Create directory structure
        self.create_directory_structure()

        # Create enhanced UI
        self.create_enhanced_ui()

        print("\n🎯 Enhanced Download System Ready!")
        print("✨ Features enabled:")
        print("  🔐 HuggingFace authentication support")
        print("  🎨 Rich interactive UI with progress tracking")
        print("  ⚡ Multiple download methods with fallback")
        print("  📊 Real-time statistics and progress monitoring")
        print("  🔄 Resume capability for interrupted downloads")

        return True

# Colab function bindings for JavaScript integration
def authenticate_hf(token):
    """Authenticate with HuggingFace (called from JavaScript)"""
    global downloader
    if downloader.hf_auth.authenticate(token):
        print("🔑 HuggingFace authentication successful!")
    else:
        print("❌ HuggingFace authentication failed!")

def skip_hf_auth():
    """Skip HuggingFace authentication (called from JavaScript)"""
    print("⏭️ Skipped HuggingFace authentication - using public access only")

def download_model(model_key):
    """Download specific model (called from JavaScript)"""
    global downloader
    print(f"🚀 Starting download of {model_key}...")
    # Implementation for downloading specific model

def download_all_essential():
    """Download all essential components (called from JavaScript)"""
    global downloader
    print("🚀 Starting download of all essential components...")
    # Implementation for downloading all essential files

def download_models_only():
    """Download models only (called from JavaScript)"""
    global downloader
    print("🎬 Starting download of models only...")
    # Implementation for downloading models only

def download_loras_only():
    """Download LoRAs only (called from JavaScript)"""
    global downloader
    print("⚡ Starting download of LoRAs only...")
    # Implementation for downloading LoRAs only

# Execute the enhanced download system
downloader = EnhancedModelDownloader()
downloader.run_enhanced_download_system()


In [8]:
#@title Cell 4 - WAN2GP Launch (Cross-Platform Final v6.0)

import sys
import os
import subprocess
import time
from IPython.display import display, HTML

def jupyter_lab_title(title):
    if 'google.colab' not in sys.modules:
        from IPython.display import display, Markdown
        display(Markdown(f"## {title}"))

jupyter_lab_title("WAN2GP Launch (Cross-Platform Final)")

class CrossPlatformWAN2GPLauncher:
    def __init__(self):
        # Use global platform configuration
        try:
            self.current_platform = current_platform
            self.pip_cmd = pip_cmd
            self.python_cmd = python_cmd
            self.use_venv = use_venv
        except NameError:
            print("⚠️ Platform detection not found. Using fallback.")
            if 'google.colab' in sys.modules:
                self.current_platform = "Google Colab"
                self.pip_cmd, self.python_cmd, self.use_venv = ".venv/bin/pip", ".venv/bin/python", True
            else:
                self.current_platform = "Lightning AI"
                self.pip_cmd, self.python_cmd, self.use_venv = "pip", "python", False

        self.process = None
        self.launch_success = False

    def verify_environment(self):
        """Verify the environment setup"""
        print(f"\n🔍 Verifying Environment Setup")
        print("=" * 50)
        print(f"📋 Platform: {self.current_platform}")
        print(f"📋 Python Command: {self.python_cmd}")
        print(f"📋 Pip Command: {self.pip_cmd}")
        print(f"📋 Virtual Environment: {'Yes' if self.use_venv else 'No (Lightning AI)'}")

        # Verify Python executable exists
        if self.use_venv and not os.path.exists(self.python_cmd):
            print(f"❌ Virtual environment Python not found: {self.python_cmd}")
            return False

        # Test basic Python functionality
        try:
            result = subprocess.run([
                self.python_cmd, '-c', 'import sys; print(f"Python: {sys.version}"); print(f"Executable: {sys.executable}")'
            ], capture_output=True, text=True, timeout=15)

            if result.returncode == 0:
                print("✅ Python Environment Verified:")
                for line in result.stdout.strip().split('\n'):
                    if line.strip():
                        print(f"   {line}")
                return True
            else:
                print(f"❌ Python verification failed: {result.stderr}")
                return False
        except Exception as e:
            print(f"❌ Python verification error: {e}")
            return False

    def verify_pytorch(self):
        """Verify PyTorch installation"""
        print(f"\n🔥 PyTorch Verification")
        print("-" * 30)

        pytorch_test = '''
import torch
import sys
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
print(f"✅ CUDA version: {torch.version.cuda}")
if torch.cuda.is_available():
    print(f"✅ GPU device: {torch.cuda.get_device_name(0)}")
print(f"✅ uint16 support: {hasattr(torch, 'uint16')}")
print(f"✅ uint32 support: {hasattr(torch, 'uint32')}")
print(f"✅ Python executable: {sys.executable}")
'''

        try:
            result = subprocess.run([
                self.python_cmd, '-c', pytorch_test
            ], capture_output=True, text=True, timeout=30)

            if result.returncode == 0:
                print("📋 PyTorch Status:")
                for line in result.stdout.strip().split('\n'):
                    if line.strip():
                        print(f"   {line}")

                # Check for correct PyTorch version
                if "2.6.0" in result.stdout:
                    print("\n✅ Correct PyTorch version 2.6.0 detected")
                    return True
                else:
                    print("\n⚠️ PyTorch version mismatch detected")
                    return False
            else:
                print(f"❌ PyTorch verification failed: {result.stderr}")
                return False
        except Exception as e:
            print(f"❌ PyTorch verification error: {e}")
            return False

    def test_wan2gp_imports(self):
        """Test WAN2GP imports"""
        print(f"\n🧪 Testing WAN2GP Imports")
        print("-" * 30)

        import_test = '''
import os
os.environ["MPLBACKEND"] = "Agg"  # Set safe matplotlib backend

try:
    from mmgp import offload, safetensors2, profile_type
    print("✅ WAN2GP core modules imported successfully")
    print("✅ mmgp.offload: Available")
    print("✅ mmgp.safetensors2: Available")
    print("✅ mmgp.profile_type: Available")
except Exception as e:
    print(f"❌ WAN2GP import error: {e}")
    exit(1)

print("✅ All WAN2GP imports successful - ready to launch")
'''

        try:
            result = subprocess.run([
                self.python_cmd, '-c', import_test
            ], capture_output=True, text=True, timeout=60)

            if result.returncode == 0:
                print("📋 Import Test Results:")
                for line in result.stdout.strip().split('\n'):
                    if line.strip():
                        print(f"   {line}")
                return True
            else:
                print(f"❌ WAN2GP import test failed:")
                for line in result.stderr.strip().split('\n'):
                    if line.strip():
                        print(f"   {line}")
                return False
        except Exception as e:
            print(f"❌ Import test error: {e}")
            return False

    def launch_wan2gp(self):
        """Launch WAN2GP with platform-specific configuration"""
        print(f"\n🚀 Launching WAN2GP on {self.current_platform}")
        print("=" * 60)

        # Platform-specific launch configuration
        base_cmd = [self.python_cmd, "wgp.py"]

        launch_args = [
            "--i2v",
            "--port", "7860",
            "--server-name", "127.0.0.1",
            "--share",
            "--attention", "sdpa",  # Most compatible
            "--profile", "4",       # Memory efficient
            "--verbose", "1"
        ]

        # Platform-specific optimizations
        if self.current_platform == "Lightning AI":
            print("⚡ Lightning AI configuration applied")
        elif self.current_platform == "Google Colab":
            launch_args.extend(["--lora-dir", "/content/Wan2GP/loras"])
            print("📱 Google Colab configuration applied")

        launch_cmd = base_cmd + launch_args

        print(f"📋 Launch Command: {' '.join(launch_cmd)}")
        print(f"🔗 Server Port: 7860")
        print(f"🐍 Python: {self.python_cmd}")
        print(f"📋 Platform: {self.current_platform}")

        # Set environment for clean launch
        launch_env = os.environ.copy()
        launch_env["MPLBACKEND"] = "Agg"

        if self.current_platform == "Google Colab":
            print("\n📱 Google Colab Access Instructions:")
            print("   1. Look for 'Running on public URL: https://...' in output below")
            print("   2. Click the public URL to access WAN2GP interface")
        elif self.current_platform == "Lightning AI":
            print("\n⚡ Lightning AI Access Instructions:")
            print("   1. Look for 'Running on local URL: http://...' in output below")
            print("   2. Use Lightning AI's port forwarding for external access")

        print("\n" + "=" * 60)
        print("🔄 Starting WAN2GP server...")
        print("💡 This may take 1-3 minutes for initial model loading")
        print("=" * 60)

        try:
            self.process = subprocess.Popen(
                launch_cmd,
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                universal_newlines=True,
                bufsize=1,
                cwd=os.getcwd(),
                env=launch_env
            )

            startup_timeout = 180
            start_time = time.time()

            while True:
                if self.process.poll() is not None:
                    return_code = self.process.returncode
                    if return_code != 0:
                        print(f"\n❌ WAN2GP process exited with error code: {return_code}")
                        return False
                    else:
                        print(f"\n✅ WAN2GP process completed successfully")
                        return True

                elapsed = time.time() - start_time
                if elapsed > startup_timeout:
                    print(f"\n⏰ Startup timeout reached ({startup_timeout}s)")
                    return True

                try:
                    line = self.process.stdout.readline()
                    if line:
                        line = line.rstrip()
                        print(line)

                        if "Running on public URL:" in line or "Running on local URL:" in line:
                            print("\n🎉 WAN2GP server started successfully!")
                            print("🌐 Use the URLs above to access the interface")
                            self.launch_success = True
                            return True

                        if "CUDA out of memory" in line:
                            print("\n💥 CUDA memory error - try smaller model")
                            return False

                except Exception as e:
                    print(f"Output reading error: {e}")
                    break

                time.sleep(0.1)

            return False

        except Exception as e:
            print(f"\n💥 Launch failed: {e}")
            return False

    def run_complete_launch(self):
        """Execute complete launch sequence"""
        print("🔧 WAN2GP Complete Launch Sequence")
        print("=" * 60)

        # Step 1: Environment verification
        if not self.verify_environment():
            print("\n💥 Environment verification failed")
            return False

        # Step 2: PyTorch verification
        if not self.verify_pytorch():
            print("\n💥 PyTorch verification failed")
            return False

        # Step 3: WAN2GP imports
        if not self.test_wan2gp_imports():
            print("\n💥 WAN2GP import test failed")
            return False

        # Step 4: Launch WAN2GP
        launch_success = self.launch_wan2gp()

        if launch_success:
            print(f"\n🎯 WAN2GP Launch Complete!")
            print("=" * 60)
            print("✅ Status: Successfully launched")
            print(f"✅ Platform: {self.current_platform}")
            print(f"✅ Environment: {'Virtual Environment' if self.use_venv else 'System (Lightning AI)'}")
            print("✅ Server: Running on port 7860")

            if self.current_platform == "Google Colab":
                print("\n📱 Next Steps:")
                print("   • Click the public URL above to access WAN2GP")
                print("   • Start with Wan 2.1 T2V 1.3B model for testing")
            elif self.current_platform == "Lightning AI":
                print("\n⚡ Next Steps:")
                print("   • Use Lightning AI port forwarding for external access")
                print("   • System environment used as required")

            return True
        else:
            print("\n💥 WAN2GP launch failed")
            return False

# Create launcher and display platform info
launcher = CrossPlatformWAN2GPLauncher()

display(HTML(f"""
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 20px; border-radius: 12px; margin: 15px 0;">
    <h3 style="margin: 0 0 15px 0;">🚀 WAN2GP Cross-Platform Launcher</h3>
    <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 8px;">
        <p style="margin: 0 0 10px 0;"><strong>Platform:</strong> {launcher.current_platform}</p>
        <p style="margin: 0 0 10px 0;"><strong>Environment:</strong> {'Virtual Environment' if launcher.use_venv else 'System (Lightning AI compliant)'}</p>
        <p style="margin: 0 0 10px 0;"><strong>Python:</strong> {launcher.python_cmd}</p>
        <p style="margin: 0;"><strong>Configuration:</strong> Cross-platform optimized with automatic platform detection</p>
    </div>
</div>
"""))

# Execute launch sequence
success = launcher.run_complete_launch()

if success:
    display(HTML("""
    <div style="background-color: #d4edda; border: 1px solid #c3e6cb; color: #155724; padding: 15px; border-radius: 8px; margin: 15px 0;">
        <h4 style="margin: 0 0 10px 0;">✅ WAN2GP Successfully Launched!</h4>
        <p style="margin: 0;">The WAN2GP video generation interface is now running with proper cross-platform configuration.</p>
    </div>
    """))
else:
    display(HTML(f"""
    <div style="background-color: #f8d7da; border: 1px solid #f5c6cb; color: #721c24; padding: 15px; border-radius: 8px; margin: 15px 0;">
        <h4 style="margin: 0 0 10px 0;">⚠️ Launch Failed</h4>
        <p style="margin: 0 0 10px 0;">Platform: {launcher.current_platform}</p>
        <p style="margin: 0;">Please check the error output above and try the emergency launch command:</p>
        <code style="background: #f1f1f1; padding: 5px; border-radius: 3px; display: block; margin: 5px 0;">
        {launcher.python_cmd} wgp.py --attention sdpa --profile 4
        </code>
    </div>
    """))

print("\n✨ Cross-platform Cell 4 execution complete!")


🔧 WAN2GP Complete Launch Sequence

🔍 Verifying Environment Setup
📋 Platform: Google Colab
📋 Python Command: .venv/bin/python
📋 Pip Command: .venv/bin/pip
📋 Virtual Environment: Yes
✅ Python Environment Verified:
   Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
   Executable: /content/Wan2GP/Wan2GP/.venv/bin/python

🔥 PyTorch Verification
------------------------------
❌ PyTorch verification failed: 

💥 PyTorch verification failed



✨ Cross-platform Cell 4 execution complete!


In [ ]:
#@title Cell (1) - WAN2GP Comprehensive Diagnostic and Auto-Repair System (Latest v6.2 Compatible)

import sys
import os
import subprocess
import shutil
import time
import json
from pathlib import Path
from IPython.display import display, HTML, Markdown
import importlib.util

class WAN2GPDiagnosticAndRepair:
    def __init__(self):
        self.platform = self.detect_platform()
        self.pip_cmd, self.python_cmd, self.use_venv = self.get_platform_commands()
        self.issues_found = []
        self.repairs_applied = []
        self.gpu_info = {}

    def detect_platform(self):
        """Enhanced platform detection with comprehensive indicators"""
        # Lightning AI detection - multiple indicators for reliability
        lightning_indicators = [
            "lightning" in str(sys.executable).lower(),
            "teamspace-studios" in os.getcwd(),
            "LIGHTNING_CLOUDSPACE_HOST" in os.environ,
            "LIGHTNING_CLOUDSPACE_ID" in os.environ,
            "/commands/python" in str(sys.executable),
            "/home/zeus/miniconda3/envs/cloudspace" in str(sys.executable),
            os.path.exists("/teamspace"),
            os.path.exists("/commands")
        ]

        # Google Colab detection
        colab_indicators = [
            "google.colab" in sys.modules,
            "/content" in os.getcwd()
        ]

        # Vast.AI detection
        vast_indicators = [
            "VAST_CONTAINER_LABEL" in os.environ,
            "/workspace" in os.getcwd(),
            "vast" in os.environ.get("HOSTNAME", "").lower()
        ]

        if any(lightning_indicators):
            return "Lightning AI"
        elif any(colab_indicators):
            return "Google Colab"
        elif any(vast_indicators):
            return "Vast.AI/Generic"
        else:
            return "Vast.AI/Generic"

    def get_platform_commands(self):
        """Get platform-specific pip and python commands"""
        if self.platform == "Lightning AI":
            return "pip", "python", False  # (pip_cmd, python_cmd, use_venv)
        elif self.platform == "Google Colab":
            return ".venv/bin/pip", ".venv/bin/python", True
        else:  # Vast.AI/Generic
            return ".venv/bin/pip", ".venv/bin/python", True

    def run_command_safely(self, command, description, timeout=60):
        """Execute command with comprehensive error handling"""
        try:
            result = subprocess.run(
                command, capture_output=True, text=True, timeout=timeout, shell=isinstance(command, str)
            )
            if result.returncode == 0:
                return True, result.stdout
            else:
                return False, result.stderr
        except subprocess.TimeoutExpired:
            return False, f"Timeout after {timeout}s"
        except Exception as e:
            return False, str(e)

    def check_gpu_compatibility(self):
        """Comprehensive GPU detection and compatibility check"""
        print("🔍 Checking GPU compatibility...")

        # Check NVIDIA GPU presence
        success, output = self.run_command_safely("nvidia-smi", "GPU Detection")
        if not success:
            self.issues_found.append("No NVIDIA GPU detected or nvidia-smi not available")
            return False

        # Parse GPU info
        try:
            # Extract GPU name and VRAM from nvidia-smi output
            lines = output.split('\n')
            for line in lines:
                if 'RTX' in line or 'GTX' in line or 'Tesla' in line or 'A100' in line:
                    gpu_name = line.split('|')[1].strip() if '|' in line else "Unknown GPU"
                    self.gpu_info['name'] = gpu_name
                    break

            # Check VRAM
            for line in lines:
                if 'MiB' in line and '/' in line:
                    vram_info = [part for part in line.split() if 'MiB' in part]
                    if len(vram_info) >= 2:
                        total_vram = int(vram_info[-1].replace('MiB', ''))
                        self.gpu_info['vram_mb'] = total_vram
                        self.gpu_info['vram_gb'] = total_vram / 1024
                        break

            print(f"✅ GPU detected: {self.gpu_info.get('name', 'Unknown')}")
            print(f"✅ VRAM: {self.gpu_info.get('vram_gb', 0):.1f}GB")

            # Check VRAM adequacy
            vram_gb = self.gpu_info.get('vram_gb', 0)
            if vram_gb < 6:
                self.issues_found.append(f"Low VRAM detected ({vram_gb:.1f}GB). Minimum 6GB recommended.")

            return True

        except Exception as e:
            self.issues_found.append(f"GPU info parsing failed: {str(e)}")
            return False

    def check_cuda_pytorch_compatibility(self):
        """Check CUDA and PyTorch compatibility"""
        print("🔍 Checking CUDA and PyTorch compatibility...")

        # Check CUDA version
        success, cuda_output = self.run_command_safely("nvcc --version", "CUDA Version Check")
        if not success:
            success, cuda_output = self.run_command_safely("nvidia-smi", "CUDA Runtime Check")

        # Check PyTorch installation
        try:
            import torch
            pytorch_version = torch.__version__
            cuda_available = torch.cuda.is_available()

            print(f"✅ PyTorch version: {pytorch_version}")
            print(f"✅ CUDA available in PyTorch: {cuda_available}")

            if not cuda_available:
                self.issues_found.append("PyTorch cannot detect CUDA")
                return False

            # Check for version compatibility
            if "50" in self.gpu_info.get('name', '') and not pytorch_version.startswith('2.7'):
                self.issues_found.append("RTX 50XX series requires PyTorch 2.7.0 or newer")

            return True

        except ImportError:
            self.issues_found.append("PyTorch not installed")
            return False
        except Exception as e:
            self.issues_found.append(f"PyTorch check failed: {str(e)}")
            return False

    def check_python_version(self):
        """Check Python version compatibility"""
        print("🔍 Checking Python version...")

        python_version = sys.version_info
        version_string = f"{python_version.major}.{python_version.minor}.{python_version.micro}"
        print(f"✅ Python version: {version_string}")

        if python_version.major != 3 or python_version.minor != 10:
            self.issues_found.append(f"Python {version_string} detected. Python 3.10.9 recommended for best compatibility.")

        return True

    def check_wan2gp_installation(self):
        """Check WAN2GP installation and repository"""
        print("🔍 Checking WAN2GP installation...")

        wan2gp_paths = ["Wan2GP", "wan2gp", "WAN2GP", "../Wan2GP", "../wan2gp"]
        wan2gp_found = False

        for path in wan2gp_paths:
            if os.path.exists(path):
                wan2gp_found = True
                wgp_py_path = os.path.join(path, "wgp.py")
                if os.path.exists(wgp_py_path):
                    print(f"✅ WAN2GP found at: {path}")
                    return True
                break

        if not wan2gp_found:
            self.issues_found.append("WAN2GP repository not found")
            return False

        self.issues_found.append("WAN2GP repository found but wgp.py missing")
        return False

    def check_dependencies(self):
        """Check critical dependencies"""
        print("🔍 Checking critical dependencies...")

        critical_deps = {
            'torch': 'PyTorch',
            'torchvision': 'TorchVision',
            'gradio': 'Gradio',
            'transformers': 'Transformers',
            'accelerate': 'Accelerate',
            'diffusers': 'Diffusers'
        }

        missing_deps = []
        for dep, name in critical_deps.items():
            try:
                importlib.import_module(dep)
                print(f"✅ {name} installed")
            except ImportError:
                missing_deps.append(dep)
                print(f"❌ {name} missing")

        if missing_deps:
            self.issues_found.append(f"Missing dependencies: {', '.join(missing_deps)}")
            return False

        return True

    def check_attention_mechanisms(self):
        """Check available attention mechanisms"""
        print("🔍 Checking attention mechanisms...")

        attention_status = {}

        # Check Triton
        try:
            import triton
            attention_status['triton'] = f"✅ Triton {triton.__version__}"
        except ImportError:
            attention_status['triton'] = "❌ Triton not available"

        # Check SageAttention
        try:
            import sageattention
            attention_status['sage'] = "✅ SageAttention available"
        except ImportError:
            attention_status['sage'] = "❌ SageAttention not available"

        # Check Flash Attention
        try:
            import flash_attn
            attention_status['flash'] = "✅ Flash Attention available"
        except ImportError:
            attention_status['flash'] = "❌ Flash Attention not available"

        for mech, status in attention_status.items():
            print(f"  {status}")

        # Recommend fallback if advanced attention not available
        if all("❌" in status for status in attention_status.values()):
            self.issues_found.append("No advanced attention mechanisms available. Will use SDPA fallback.")

        return True

    def check_memory_and_performance(self):
        """Check memory and performance configuration"""
        print("🔍 Checking memory and performance...")

        # Check available system RAM
        try:
            import psutil
            ram_gb = psutil.virtual_memory().total / (1024**3)
            print(f"✅ System RAM: {ram_gb:.1f}GB")

            if ram_gb < 16:
                self.issues_found.append(f"Low system RAM ({ram_gb:.1f}GB). 16GB+ recommended for best performance.")
        except ImportError:
            print("⚠️ Cannot check system RAM (psutil not available)")

        # Check disk space
        try:
            disk_usage = shutil.disk_usage(os.getcwd())
            free_gb = disk_usage.free / (1024**3)
            print(f"✅ Free disk space: {free_gb:.1f}GB")

            if free_gb < 20:
                self.issues_found.append(f"Low disk space ({free_gb:.1f}GB). 20GB+ recommended.")
        except Exception:
            print("⚠️ Cannot check disk space")

        return True

    def auto_repair_pytorch(self):
        """Auto-repair PyTorch installation"""
        print("🔧 Attempting PyTorch repair...")

        # Determine correct PyTorch version based on GPU
        if "50" in self.gpu_info.get('name', ''):
            # RTX 50XX series
            pytorch_cmd = [
                self.pip_cmd, "install", "--upgrade",
                "torch==2.7.0", "torchvision", "torchaudio",
                "--index-url", "https://download.pytorch.org/whl/test/cu128"
            ]
        else:
            # RTX 10XX-40XX series
            pytorch_cmd = [
                self.pip_cmd, "install", "--upgrade",
                "torch==2.6.0", "torchvision", "torchaudio",
                "--index-url", "https://download.pytorch.org/whl/test/cu124"
            ]

        success, output = self.run_command_safely(pytorch_cmd, "PyTorch Installation", timeout=300)
        if success:
            self.repairs_applied.append("PyTorch installation repaired")
            return True
        else:
            print(f"❌ PyTorch repair failed: {output}")
            return False

    def auto_repair_dependencies(self):
        """Auto-repair missing dependencies"""
        print("🔧 Installing missing dependencies...")

        # Install core requirements
        requirements_cmd = [self.pip_cmd, "install", "-r", "requirements.txt"]
        success, output = self.run_command_safely(requirements_cmd, "Dependencies Installation", timeout=300)

        if success:
            self.repairs_applied.append("Dependencies installed")
            return True
        else:
            # Fallback: install critical packages individually
            critical_packages = [
                "gradio>=4.0.0", "transformers", "accelerate", "diffusers",
                "opencv-python", "Pillow", "numpy", "scipy"
            ]

            for package in critical_packages:
                cmd = [self.pip_cmd, "install", package]
                success, _ = self.run_command_safely(cmd, f"Installing {package}", timeout=60)
                if success:
                    print(f"✅ Installed {package}")

            self.repairs_applied.append("Critical dependencies installed individually")
            return True

    def auto_repair_wan2gp_repo(self):
        """Auto-repair WAN2GP repository"""
        print("🔧 Cloning WAN2GP repository...")

        repo_url = "https://github.com/deepbeepmeep/Wan2GP.git"
        clone_cmd = ["git", "clone", "--depth", "1", repo_url]

        success, output = self.run_command_safely(clone_cmd, "Repository Clone", timeout=120)
        if success:
            self.repairs_applied.append("WAN2GP repository cloned")
            return True
        else:
            print(f"❌ Repository clone failed: {output}")
            return False

    def auto_repair_attention_mechanisms(self):
        """Auto-repair attention mechanisms"""
        print("🔧 Installing performance optimizations...")

        repairs = []

        # Install Triton for Windows
        if os.name == 'nt':  # Windows
            triton_cmd = [self.pip_cmd, "install", "triton-windows"]
            success, _ = self.run_command_safely(triton_cmd, "Triton Installation", timeout=120)
            if success:
                repairs.append("Triton (Windows)")

        # Install SageAttention
        sage_cmd = [self.pip_cmd, "install", "sageattention>=1.0.6"]
        success, _ = self.run_command_safely(sage_cmd, "SageAttention Installation", timeout=120)
        if success:
            repairs.append("SageAttention")

        if repairs:
            self.repairs_applied.append(f"Installed: {', '.join(repairs)}")
            return True

        return False

    def generate_optimized_launch_commands(self):
        """Generate optimized launch commands based on hardware"""
        print("\n🚀 Generating optimized launch commands...")

        vram_gb = self.gpu_info.get('vram_gb', 8)
        gpu_name = self.gpu_info.get('name', 'Unknown')

        commands = {}

        # Base command components
        base_cmd = "python wgp.py"

        if vram_gb < 8:
            # Low VRAM setup
            commands['Low VRAM (6-8GB)'] = f"{base_cmd} --t2v-1-3B --attention sdpa --profile 4 --teacache 1.5"
        elif vram_gb < 12:
            # Medium VRAM setup
            commands['Medium VRAM (8-12GB)'] = f"{base_cmd} --t2v-14B --attention sage --profile 4 --teacache 2.0"
        else:
            # High VRAM setup
            commands['High VRAM (12GB+)'] = f"{base_cmd} --t2v-14B --attention sage2 --profile 3 --compile --teacache 2.0"

        # GPU-specific optimizations
        if "10" in gpu_name or "20" in gpu_name:
            commands['RTX 10XX/20XX Optimized'] = f"{base_cmd} --attention sdpa --profile 4 --teacache 1.5"
        elif "30" in gpu_name or "40" in gpu_name:
            commands['RTX 30XX/40XX Optimized'] = f"{base_cmd} --compile --attention sage --profile 3 --teacache 2.0"
        elif "50" in gpu_name:
            commands['RTX 50XX Optimized'] = f"{base_cmd} --attention sage --profile 4 --fp16"

        # Fallback command
        commands['Safe Fallback'] = f"{base_cmd} --t2v-1-3B --attention sdpa --profile 4 --teacache 0 --fp16"

        # Debug command
        commands['Debug Mode'] = f"{base_cmd} --verbose 2 --check-loras --attention sdpa --profile 4"

        return commands

    def run_full_diagnostic(self):
        """Run complete diagnostic and repair sequence"""
        print("🏥 WAN2GP Comprehensive Diagnostic and Auto-Repair System")
        print("=" * 60)
        print(f"🖥️ Platform: {self.platform}")
        print(f"🐍 Python: {self.python_cmd}")
        print(f"📦 Pip: {self.pip_cmd}")
        print(f"🔧 Virtual Environment: {'Yes' if self.use_venv else 'No'}")
        print("=" * 60)

        # Run all diagnostics
        checks = [
            self.check_python_version,
            self.check_gpu_compatibility,
            self.check_cuda_pytorch_compatibility,
            self.check_wan2gp_installation,
            self.check_dependencies,
            self.check_attention_mechanisms,
            self.check_memory_and_performance
        ]

        print("\n📋 Running Diagnostics...")
        for check in checks:
            try:
                check()
            except Exception as e:
                print(f"❌ Diagnostic error: {str(e)}")
                self.issues_found.append(f"Diagnostic error: {str(e)}")

        # Auto-repair if issues found
        if self.issues_found:
            print(f"\n⚠️ Found {len(self.issues_found)} issues:")
            for i, issue in enumerate(self.issues_found, 1):
                print(f"  {i}. {issue}")

            print(f"\n🔧 Attempting automatic repairs...")

            # Apply repairs based on issues found
            if any("PyTorch" in issue for issue in self.issues_found):
                self.auto_repair_pytorch()

            if any("dependencies" in issue.lower() for issue in self.issues_found):
                self.auto_repair_dependencies()

            if any("repository" in issue.lower() for issue in self.issues_found):
                self.auto_repair_wan2gp_repo()

            if any("attention" in issue.lower() for issue in self.issues_found):
                self.auto_repair_attention_mechanisms()

        # Generate launch commands
        commands = self.generate_optimized_launch_commands()

        # Final report
        self.display_final_report(commands)

    def display_final_report(self, commands):
        """Display comprehensive final report"""

        # Create styled HTML report
        html_report = f"""
        <div style="font-family: Arial, sans-serif; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    color: white; padding: 20px; border-radius: 10px; margin: 10px 0;">
            <h2>🏥 WAN2GP Diagnostic Report</h2>
            <div style="background-color: rgba(255,255,255,0.1); padding: 15px; border-radius: 8px; margin: 10px 0;">
                <h3>📊 System Status</h3>
                <p><strong>Platform:</strong> {self.platform}</p>
                <p><strong>GPU:</strong> {self.gpu_info.get('name', 'Unknown')} ({self.gpu_info.get('vram_gb', 0):.1f}GB VRAM)</p>
                <p><strong>Issues Found:</strong> {len(self.issues_found)}</p>
                <p><strong>Repairs Applied:</strong> {len(self.repairs_applied)}</p>
            </div>
        </div>
        """

        if self.repairs_applied:
            html_report += f"""
            <div style="background-color: #28a745; color: white; padding: 15px; border-radius: 8px; margin: 10px 0;">
                <h3>✅ Repairs Applied Successfully:</h3>
                <ul>
            """
            for repair in self.repairs_applied:
                html_report += f"<li>{repair}</li>"
            html_report += "</ul></div>"

        if self.issues_found and not self.repairs_applied:
            html_report += f"""
            <div style="background-color: #dc3545; color: white; padding: 15px; border-radius: 8px; margin: 10px 0;">
                <h3>⚠️ Unresolved Issues:</h3>
                <ul>
            """
            for issue in self.issues_found:
                html_report += f"<li>{issue}</li>"
            html_report += "</ul></div>"

        display(HTML(html_report))

        # Display optimized commands
        print("\n🚀 Recommended Launch Commands:")
        print("=" * 50)
        for name, command in commands.items():
            print(f"\n{name}:")
            print(f"  {command}")

        print("\n" + "=" * 50)
        print("✅ Diagnostic complete! Use the appropriate command above to launch WAN2GP.")
        if self.issues_found and not self.repairs_applied:
            print("⚠️  Some issues require manual intervention. Check the report above.")

# Execute diagnostic system
diagnostic_system = WAN2GPDiagnosticAndRepair()
diagnostic_system.run_full_diagnostic()


In [ ]:
#@title Cell (2) - Quick Diagnostic Runner and Emergency Repair Tools (Latest v6.2)

import subprocess
import sys
import os
from IPython.display import display, HTML

class QuickDiagnosticRunner:
    def __init__(self):
        self.platform = self.detect_platform()
        self.pip_cmd, self.python_cmd = self.get_platform_commands()

    def detect_platform(self):
        """Quick platform detection"""
        if "google.colab" in sys.modules:
            return "Google Colab"
        elif any(indicator in str(sys.executable).lower() for indicator in ["lightning", "teamspace"]):
            return "Lightning AI"
        else:
            return "Generic/Vast.AI"

    def get_platform_commands(self):
        """Get platform-specific commands"""
        if self.platform == "Lightning AI":
            return "pip", "python"
        else:
            return ".venv/bin/pip" if os.path.exists(".venv") else "pip", ".venv/bin/python" if os.path.exists(".venv") else "python"

    def run_cmd(self, cmd, timeout=30):
        """Execute command safely"""
        try:
            result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=timeout)
            return result.returncode == 0, result.stdout, result.stderr
        except subprocess.TimeoutExpired:
            return False, "", "Timeout"
        except Exception as e:
            return False, "", str(e)

    def emergency_pytorch_fix(self):
        """Emergency PyTorch installation fix"""
        print("🚨 Emergency PyTorch Fix...")

        # Detect GPU generation for correct PyTorch version
        success, gpu_info, _ = self.run_cmd("nvidia-smi")

        if "RTX 50" in gpu_info:
            pytorch_url = "https://download.pytorch.org/whl/test/cu128"
            torch_version = "torch==2.7.0"
        else:
            pytorch_url = "https://download.pytorch.org/whl/test/cu124"
            torch_version = "torch==2.6.0"

        cmd = f"{self.pip_cmd} install --upgrade {torch_version} torchvision torchaudio --index-url {pytorch_url}"
        success, stdout, stderr = self.run_cmd(cmd, timeout=300)

        if success:
            print("✅ PyTorch emergency fix applied")
            return True
        else:
            print(f"❌ PyTorch fix failed: {stderr}")
            return False

    def emergency_dependency_fix(self):
        """Emergency dependency installation"""
        print("🚨 Emergency Dependency Fix...")

        essential_packages = [
            "gradio>=4.0.0", "transformers", "accelerate", "diffusers",
            "opencv-python", "Pillow", "numpy", "scipy", "psutil"
        ]

        for package in essential_packages:
            cmd = f"{self.pip_cmd} install {package}"
            success, _, _ = self.run_cmd(cmd, timeout=60)
            print("✅" if success else "❌", package)

        print("✅ Emergency dependencies installed")

    def quick_system_check(self):
        """Quick system health check"""
        checks = {}

        # GPU Check
        success, output, _ = self.run_cmd("nvidia-smi")
        checks['GPU'] = "✅ Available" if success and "RTX" in output else "❌ Issue detected"

        # PyTorch Check
        try:
            import torch
            checks['PyTorch'] = f"✅ {torch.__version__}" if torch.cuda.is_available() else "❌ CUDA not available"
        except ImportError:
            checks['PyTorch'] = "❌ Not installed"

        # WAN2GP Check
        wan_exists = any(os.path.exists(path) for path in ["Wan2GP/wgp.py", "wan2gp/wgp.py", "WAN2GP/wgp.py"])
        checks['WAN2GP'] = "✅ Found" if wan_exists else "❌ Missing"

        # Dependencies Check
        try:
            import gradio, transformers, accelerate, diffusers
            checks['Dependencies'] = "✅ Core packages available"
        except ImportError:
            checks['Dependencies'] = "❌ Missing packages"

        return checks

    def generate_emergency_commands(self):
        """Generate emergency launch commands"""
        commands = {
            "Ultra Safe Mode": f"{self.python_cmd} wgp.py --t2v-1-3B --attention sdpa --profile 4 --teacache 0 --fp16",
            "Memory Emergency": f"{self.python_cmd} wgp.py --t2v-1-3B --profile 5 --perc-reserved-mem-max 0.2",
            "Debug Mode": f"{self.python_cmd} wgp.py --verbose 2 --attention sdpa --profile 4",
            "Network Share": f"{self.python_cmd} wgp.py --listen --server-port 7861 --attention sdpa"
        }
        return commands

    def run_quick_diagnostic(self):
        """Run quick diagnostic and provide emergency options"""

        print(f"⚡ Quick Diagnostic - Platform: {self.platform}")
        print("=" * 40)

        # Quick system check
        checks = self.quick_system_check()

        issues = []
        for component, status in checks.items():
            print(f"{component}: {status}")
            if "❌" in status:
                issues.append(component)

        # Emergency repairs
        if issues:
            print(f"\n🚨 {len(issues)} issues detected. Applying emergency fixes...")

            if "PyTorch" in issues:
                self.emergency_pytorch_fix()

            if "Dependencies" in issues:
                self.emergency_dependency_fix()

            if "WAN2GP" in issues:
                print("🔧 Cloning WAN2GP repository...")
                success, _, _ = self.run_cmd("git clone --depth 1 https://github.com/deepbeepmeep/Wan2GP.git", timeout=120)
                print("✅ Repository cloned" if success else "❌ Clone failed")

        # Generate emergency commands
        commands = self.generate_emergency_commands()

        # Display results
        html_display = f"""
        <div style="background: linear-gradient(45deg, #ff6b6b, #feca57); color: white;
                    padding: 15px; border-radius: 8px; margin: 10px 0;">
            <h3>⚡ Quick Diagnostic Results</h3>
            <p><strong>Platform:</strong> {self.platform}</p>
            <p><strong>Issues Found:</strong> {len(issues)}</p>
            <p><strong>Status:</strong> {'🚨 Needs Attention' if issues else '✅ System Ready'}</p>
        </div>
        """

        display(HTML(html_display))

        print("\n🚀 Emergency Launch Commands:")
        print("-" * 40)
        for name, command in commands.items():
            print(f"\n{name}:")
            print(f"  {command}")

        print("\n" + "=" * 40)
        print("⚡ Quick diagnostic complete!")

        if not issues:
            print("✅ System appears healthy. Try the Ultra Safe Mode command first.")
        else:
            print("🚨 Emergency fixes applied. Test with Ultra Safe Mode.")

# Execute quick diagnostic
quick_runner = QuickDiagnosticRunner()
quick_runner.run_quick_diagnostic()
